In [1]:
import os
from Bio import AlignIO

In [2]:
f = "light-Jref-clustalo.fasta"
fhIn = open(f)
aln = AlignIO.read(fhIn, "fasta")
i = 0
refs = list()
exp = list()
seq_names = list()
for record in aln:
    #print(record)
    seq_names.append(record.id)
    if record.id.startswith("IG"):
        refs.append(i)
    else:
        exp.append(i)
    i = i + 1
print("refs:", refs)
print("exp:", exp)
print("names:", seq_names)

refs: [0, 1]
exp: [2, 3, 4]
names: ['IGLJ2*01', 'IGLJ3*01', 'L1B3', 'L1D4', 'L3D4']


In [3]:
n = len(aln[:, 0])  # how many sequences?
same = [n * "c", n * "a", n * "t", n * "g", n * "-"]
print(same)

['ccccc', 'aaaaa', 'ttttt', 'ggggg', '-----']


In [4]:
# Create an empty matrix for the mutations
mutations = dict()
for e in exp:
    mutations[e] = dict()
    for r in refs:
        mutations[e][r] = list()
print(mutations)

{2: {0: [], 1: []}, 3: {0: [], 1: []}, 4: {0: [], 1: []}}


In [5]:
for i in range(aln.get_alignment_length()):
    column = aln[:, i] # get column
    
    if column in same: # skip if everything is the same
        continue

    for e in exp: # for every experiment sequence
        for r in refs: # for every reference sequence
            if column[r] != column[e] and column[r] != "-" and column[e] != "-": # check for different nucleotides, skip indels
                mut = column[r] + str(i+1) + ">" + column[e]
                mutations[e][r].append(mut)
print(mutations)

{2: {0: ['t288>g', 'g289>a', 'g291>t', 'g292>a', 'a311>g'], 1: ['t288>g', 'g289>a', 'g291>t', 'g292>a', 'a311>g']}, 3: {0: ['t288>g', 'g291>t', 'g292>a', 'a294>c', 'a311>g', 'g312>t', 'c322>t'], 1: ['t288>g', 'g291>t', 'g292>a', 'a294>c', 'a311>g', 'g312>t', 'c322>t']}, 4: {0: ['g291>c', 'g292>t', 't296>g', 'c297>t', 'c300>t', 'g312>a', 'a324>g'], 1: ['g291>c', 'g292>t', 't296>g', 'c297>t', 'c300>t', 'g312>a', 'a324>g']}}


In [6]:
# Count mutations
print("Total amount of point mutations compared to the reference sequences")
mut_count = dict()
for r in refs:
    mut_count[r] = dict()
    for e in exp:
        print(seq_names[r], seq_names[e], len(mutations[e][r]))
        mut_count[r][e] = len(mutations[e][r])

Total amount of point mutations compared to the reference sequences
IGLJ2*01 L1B3 5
IGLJ2*01 L1D4 7
IGLJ2*01 L3D4 7
IGLJ3*01 L1B3 5
IGLJ3*01 L1D4 7
IGLJ3*01 L3D4 7


In [7]:
# Create matrix for common mutations compared to the references
print("Nr of common point mutations and perc compared to total nr of mutations")
for r in refs:
    for i in range(len(exp) - 1):
        e1 = exp[i]
        for j in range(i+1, len(exp)):
            e2 = exp[j]
            nr_common_mut = len(set(mutations[e1][r]).intersection(set(mutations[e2][r])))
            perc_e1 = round(100 * nr_common_mut / mut_count[r][e1], 1)
            perc_e2 = round(100 * nr_common_mut / mut_count[r][e2], 1)
            print(seq_names[r], seq_names[e1], seq_names[e2], nr_common_mut, perc_e1, perc_e2)

Nr of common point mutations and perc compared to total nr of mutations
IGLJ2*01 L1B3 L1D4 4 80.0 57.1
IGLJ2*01 L1B3 L3D4 0 0.0 0.0
IGLJ2*01 L1D4 L3D4 0 0.0 0.0
IGLJ3*01 L1B3 L1D4 4 80.0 57.1
IGLJ3*01 L1B3 L3D4 0 0.0 0.0
IGLJ3*01 L1D4 L3D4 0 0.0 0.0
